<a href="https://colab.research.google.com/github/nilesh3030/Stroke-Prediction/blob/main/Location_attachtment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Stroke_Prediction/Data/stroke_case.csv', low_memory = False)
pd.set_option ('display.max_columns', None)
pd.set_option ('display.max_rows', None)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = df[['jaenan_sn', 'address_si', 'address_gu', 'address_dong', 'call_d', 'call_t']]

data['Address'] = data['address_si'] + ", " + data['address_gu'] + ", " +  data['address_dong']
data['Address2'] = data['address_si'] + ", " + data['address_gu']
data['Address3'] = data['address_gu']
data['Address4'] = data['address_si']

def get_locations(row):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent='MyApp')
    try:
      location = geolocator.geocode(row['Address'])
      return(location.latitude, location.longitude)
    except:
      try:
        location = geolocator.geocode(row['Address2'])
        return(location.latitude, location.longitude)
      except:
        try:
          location = geolocator.geocode(row['Address3'])
          return(location.latitude, location.longitude)
        except:
          try:
            location = geolocator.geocode(row['Address4'])
            return(location.latitude, location.longitude)
          except:
            print(row['Address4'])
            return(0, 0)

In [ ]:
Address = data.groupby(['Address', 'Address2', 'Address3', 'Address4']).size().reset_index(name='Freq')

Address.head()

In [ ]:
Address['Location'] = Address.apply(lambda row: get_locations(row), axis=1)

In [ ]:
Address['latitude'], Address['longitude'] = Address.Location.str

In [ ]:
Address.head()

In [ ]:
Address.to_csv('address.csv')

In [ ]:
branch_location = pd.read_excel('/content/drive/MyDrive/Stroke_Prediction/Data/Meterological_branch_and_Location.xlsx')

In [ ]:
branch_location.head()

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist

Address["Branch Name"] = pd.Series([branch_location["Branch Name"].iloc[np.argmin(x)] for x in cdist(Address[["latitude", "longitude"]], branch_location[["Latitude", "Longitude"]])])


In [ ]:
Address.head()

In [ ]:
data = data.merge(Address, on = 'Address')

In [ ]:
data.head()

In [ ]:
data = data.merge(branch_location, on = 'Branch Name')
data.head()

In [ ]:
data.to_csv('data.csv', encoding = 'euc_kr')